import spark package for data mining

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, trim, lower, split, count, avg, format_number,isnan,sum as _sum
from functools import reduce
from pyspark.sql.types import (StructField, StringType, IntegerType, StructType, FloatType, DoubleType)
from pyspark.ml.feature import StringIndexer, OneHotEncoder, PCA, MinMaxScaler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


read dataset

In [11]:
# create sparkSession
spark = SparkSession.builder.appName("csv_example").getOrCreate()

# read CSV document
df1 = spark.read.csv('weather.csv', header=True, inferSchema=True)
df2 = spark.read.csv('fires.csv', header=True, inferSchema=True)

# print DataFrame
print("Weather DataFrame:")
df1.show()

print("Fires DataFrame:")
df2.show()

Weather DataFrame:
+---+---+---+-----+----+--------+
|num|  X|  Y|   DC| ISI| country|
+---+---+---+-----+----+--------+
|  1|  7|  5| 94.3| 5.1|Portugal|
|  2|  7|  4|669.1| 6.7|Portugal|
|  3|  7|  4|686.9| 6.7|Portugal|
|  4|  8|  6| 77.5| 9.0|Portugal|
|  5|  8|  6|102.2| 9.6|Portugal|
|  6|  8|  6|488.0|14.7|Portugal|
|  7|  8|  6|495.6| 8.5|Portugal|
|  8|  8|  6|608.2|10.7|Portugal|
|  9|  8|  6|692.6| 7.0|Portugal|
| 10|  7|  5|698.6| 7.1|Portugal|
| 11|  7|  5|698.6| 7.1|Portugal|
| 12|  7|  5|713.0|22.6|Portugal|
| 13|  6|  5|665.3| 0.8|Portugal|
| 14|  6|  5|686.5| 7.0|Portugal|
| 15|  6|  5|699.6| 9.2|Portugal|
| 16|  6|  5|713.9|13.9|Portugal|
| 17|  5|  5| 80.8| 7.8|Portugal|
| 18|  8|  5|664.2| 3.0|Portugal|
| 19|  6|  4| 70.8| 6.3|Portugal|
| 20|  6|  4| 97.1| 5.1|Portugal|
+---+---+---+-----+----+--------+
only showing top 20 rows

Fires DataFrame:
+---+---+---+-----+---+----+-----+----+---+----+----+----+---------+--------+
|num|  X|  Y|month|day|FFMC|  DMC|temp| RH|w

Find missing data,count missing rows by country and identify rows containing at least one missing value

In [12]:
# check missing value
df1.select([(col(c).isNull().cast("int")).alias(c) for c in df1.columns]).agg(*[ _sum(c).alias(c) for c in df1.columns ]).show()
df2.select([(col(c).isNull().cast("int")).alias(c) for c in df2.columns]).agg(*[ _sum(c).alias(c) for c in df2.columns ]).show()

# print the shape of DataFrame
print("Shape df1: ({}, {})".format(df1.count(), len(df1.columns)))
print("Shape df2: ({}, {})".format(df2.count(), len(df2.columns)))

# print the datatype of DataFrame
print("df1 dtype:", df1.dtypes)
print("df2 dtype:", df2.dtypes)

# integret DataFrame 1 and DataFrame 2
df3 = df1.join(df2, on=['X', 'Y', 'num', 'country'], how='outer')

# print datatype of integreted DataFrame(df3)
print("df3 dtype:", df3.dtypes)

# 计算每个国家的总行数
total_rows_per_country = df3.groupBy("country").agg(count("*").alias("total_count"))

# 计算每个国家含有缺失值的行数（检查所有列）
condition = reduce(lambda x, y: x | y, [col(c).isNull() for c in df3.columns])
missing_rows_per_country = df3.filter(condition).groupBy("country").agg(count("*").alias("missing_count"))

# 计算缺失数据的比例
missing_ratio_per_country = missing_rows_per_country.join(
    total_rows_per_country,
    on="country",
    how="inner"
).withColumn(
    "missing_ratio",
    col("missing_count") / col("total_count")
).select(
    "country",
    "missing_ratio"
).orderBy(col("missing_ratio").desc())

# 显示结果
missing_ratio_per_country.show()

+---+---+---+---+---+-------+
|num|  X|  Y| DC|ISI|country|
+---+---+---+---+---+-------+
|  0|  0|  0|  0|  0|      0|
+---+---+---+---+---+-------+

+---+---+---+-----+---+----+---+----+---+----+----+----+---------+-------+
|num|  X|  Y|month|day|FFMC|DMC|temp| RH|wind|rain|area|passenger|country|
+---+---+---+-----+---+----+---+----+---+----+----+----+---------+-------+
|  0|  0|  0|    0|  0| 369|369|   0|  0|   0|   0|   0|        0|      0|
+---+---+---+-----+---+----+---+----+---+----+----+----+---------+-------+

Shape df1: (886, 6)
Shape df2: (886, 14)
df1 dtype: [('num', 'int'), ('X', 'int'), ('Y', 'int'), ('DC', 'double'), ('ISI', 'double'), ('country', 'string')]
df2 dtype: [('num', 'int'), ('X', 'int'), ('Y', 'int'), ('month', 'string'), ('day', 'string'), ('FFMC', 'double'), ('DMC', 'double'), ('temp', 'double'), ('RH', 'int'), ('wind', 'double'), ('rain', 'double'), ('area', 'double'), ('passenger', 'int'), ('country', 'string')]
df3 dtype: [('X', 'int'), ('Y', 'int'), (

+-------+-------------+
|country|missing_ratio|
+-------+-------------+
| Brazil|          1.0|
+-------+-------------+



Visualise the rate of missing data